In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
from datasets import load_from_disk

import numpy as np
np.random.seed(42)

import json, copy, os, sys
from tqdm import tqdm

import logging
logger = logging.getLogger('logger')
logging.basicConfig(level=logging.INFO)
logger.setLevel(logging.INFO)

sys.path.append(os.path.abspath(os.getcwd()))
from annotation_utilities import *

# this is the list of phenomena and which option they need to be annotated with:
phenomena = {
    'addition':'add-omit',
    'ambiguous-translation-wrong-discourse-connective-since-causal':'diff_flexible',
    'ambiguous-translation-wrong-discourse-connective-since-temporal':'diff_flexible',
    'ambiguous-translation-wrong-discourse-connective-while-contrast':'diff_flexible',
    'ambiguous-translation-wrong-discourse-connective-while-temporal':'diff_flexible',
    'ambiguous-translation-wrong-gender-female-anti':'diff_flexible',
    'ambiguous-translation-wrong-gender-female-pro':'diff_flexible',
    'ambiguous-translation-wrong-gender-male-anti':'diff_flexible',
    'ambiguous-translation-wrong-gender-male-pro':'diff_flexible',
    'ambiguous-translation-wrong-sense-frequent':'diff_flexible',
    'ambiguous-translation-wrong-sense-infrequent':'diff_flexible',
    'anaphoric_group_it-they:deletion':'annotate_word',
    'anaphoric_group_it-they:substitution':'annotate_word',
    'anaphoric_intra_non-subject_it:deletion':'annotate_word',
    'anaphoric_intra_non-subject_it:substitution':'annotate_word',
    'anaphoric_intra_subject_it:deletion':'annotate_word',
    'anaphoric_intra_subject_it:substitution':'annotate_word',
    'anaphoric_intra_they:deletion':'annotate_word',
    'anaphoric_intra_they:substitution':'annotate_word',
    'anaphoric_singular_they:deletion':'annotate_word',
    'anaphoric_singular_they:substitution':'annotate_word',
    'antonym-replacement':'REF_flexible',
    'commonsense-only-ref-ambiguous':'diff_flexible',
    'commonsense-src-and-ref-ambiguous':'diff_flexible',
    'copy-source':'whole_sentence',
    'coreference-based-on-commonsense':'mixed_flexible',
    'do-not-translate':'whole_sentence',
    'hallucination-date-time':'date',
    'hallucination-named-entity-level-1':'diff_flexible',
    'hallucination-named-entity-level-2':'REF_flexible',
    'hallucination-named-entity-level-3':'REF_flexible',
    'hallucination-number-level-1':'diff_flexible',
    'hallucination-number-level-2':'REF_flexible',
    'hallucination-number-level-3':'REF_flexible',
    'hallucination-real-data-vs-ref-word':'diff_flexible',
    'hallucination-real-data-vs-synonym':'diff_flexible',
    'hallucination-unit-conversion-amount-matches-ref':'units',
    'hallucination-unit-conversion-unit-matches-ref':'units',
    'hypernym-replacement':'REF_flexible',
    'hyponym-replacement':'REF_flexible',
    'lexical-overlap':'?',
    'modal_verb:deletion':'add-omit',
    'modal_verb:substitution':'diff_flexible',
    'nonsense':'REF_flexible',
    'omission':'add-omit',
    'ordering-mismatch':'swap',
    'overly-literal-vs-correct-idiom':'diff_flexible',
    'overly-literal-vs-explanation':'diff_flexible',
    'overly-literal-vs-ref-word':'diff_flexible',
    'overly-literal-vs-synonym':'diff_flexible',
    'pleonastic_it:deletion':'annotate_word',
    'pleonastic_it:substitution':'annotate_word',
    'punctuation:deletion_all':'add-omit',
    'punctuation:deletion_commas':'add-omit',
    'punctuation:deletion_quotes':'add-omit',
    'punctuation:statement-to-question':'add-omit',
    'real-world-knowledge-entailment':'diff_flexible',
    'real-world-knowledge-hypernym-vs-distractor':'diff_flexible',
    'real-world-knowledge-hypernym-vs-hyponym':'diff_flexible',
    'real-world-knowledge-synonym-vs-antonym':'diff_flexible',
    'similar-language-high':'whole_sentence',
    'similar-language-low':'whole_sentence',
    'untranslated-vs-ref-word':'diff_flexible',   # here add-omit can be used for getting character level replacements too
    'untranslated-vs-synonym':'diff_flexible',
    'xnli-addition-contradiction':'?',
    'xnli-addition-neutral':'?',
    'xnli-omission-contradiction':'?',
    'xnli-omission-neutral':'?'
}

folder = os.getcwd()
manual_annotations = os.path.join(folder, 'manual_annotations')
if not os.path.exists(manual_annotations):
    os.mkdir(manual_annotations)
    
phenomena_tobe_processed = input("enter the phenomena: ") 
if phenomena_tobe_processed == 'test':
    # load the subset.json
    dataset_path = os.path.join(manual_annotations, 'subset.json')
    if not os.path.exists(dataset_path):
        logger.error('No dataset path: %s' %(dataset_path))
        exit()
    logger.info('Loading the test dataset...')
    with open(dataset_path, "r") as f:
        samples = json.load(f)
    logger.info('Test dataset loaded.')
elif phenomena_tobe_processed not in phenomena.keys():
    logger.error("The phenomena should be one of these: {}".format(sys.argv[1], phenomena.keys()))
    exit()
else:
    dataset_path = os.path.join(folder, '../../dataset')
    if not os.path.exists(dataset_path):
        logger.error('No dataset path: %s' %(dataset_path))
        exit()
    logger.info('Loading the dataset...')
    dataset = load_from_disk(dataset_path)
    logger.info('Dataset loaded.')
    samples = dict()
    for idx, sample in enumerate(dataset['train']):
        if sample['phenomena'] in phenomena_tobe_processed:
            samples[idx] = sample
        
checkpoint = os.path.join(folder, 'manual_annotations/annotated_checkpoint_{}.txt'.format(phenomena_tobe_processed))
if os.path.exists(checkpoint):
    logger.info('Path {} already exists. Loading..'.format(checkpoint))
    with open(checkpoint, "r") as f:
        annotations = json.load(f)
    annotations = {int(k):v for k,v in annotations.items()}
else:
    annotations = dict()
    
# calculate statistics about the annotations:
# for every mode, calculate no. of skipped, no. of unsure and ids, and no. of done.
stats_template = {
            'total':0,
            'success':0,
            'too_long':[],
            'no_change':[],
            'error':[],
            'other':[]  
        }
stats_path = os.path.join(folder, 'ACES_private/challenge_set_annotation/stats.txt')
if os.path.exists(stats_path):
    logger.info('Path {} already exists. Loading..'.format(stats_path))
    with open(stats_path, "r") as f:
        stats = json.load(f)
    # we want to overwrite the statistics for the new phenomena
    for p in phenomena_tobe_processed:
        stats[p] = copy.deepcopy(stats_template)
else:
    logger.info('Creating new stats.txt file at {}'.format(stats_path))
    stats = {}
    for key in phenomena.keys():
        stats[key] = copy.deepcopy(stats_template)
    stats['test'] = copy.deepcopy(stats_template)
    
            
logger.info("READY")

# the UI (?) part of the annotation in general (ask if they want to accept the annotation, call manual_annotation if no)
def manual_annotation_io(idx):
    sample = samples[idx]
    if idx in annotations:
        change = annotations[idx]['annotation']   # now it's normalized annotation.
        if len(change) == 1 and len(change[0]["in_good"]['token_index']) == len(change[0]["in_bad"]['token_index']):
            return 0
    if phenomena[sample["phenomena"]] in ['?', 'mixed_flexible']:
        print("-----> For this sample we can compare the Incorrect translation with either Reference or Good translation.")
    elif phenomena[sample["phenomena"]] in ['REF_flexible']:
        print("-----> For this sample we compare the Incorrect translation with the Reference.")
    else:
        print("-----> For this sample we compare the Incorrect translation with the Good translation.\n")
    if idx in annotations:
        print("\nID: ", idx)
        print("Source sentence: ", sample['source'])
        print("Reference: ", sample['reference'])
        print("Good Translation: ", sample['good-translation'])
        print("Incorrect Translation: ", sample['incorrect-translation'])
        print('Suggested annotation:')
        print(annotations[idx]['annotation'], '\n')
        inp = input('To accept the suggested annotation click on enter. To skip this one enter skip. To exit enter exit and enter anything else to manually annotate:')
        if inp == "skip":
            annotations.pop(idx)
            return 1  # this means, we are skipping, so should delete this annotation and then continue with the next.
        if inp == "exit":
            # do not add the annotation if you stop at this point
            annotations.pop(idx)
            return -1
        res = manual_annotation(idx, inp)
        if res == -1:
            # do not add the annotation if you stop at this point
            annotations.pop(idx)
            return -1
        if res == 1:
            # skipping
            annotations.pop(idx)
            return 1
    else:
        print("No automatic translations for this sample.")
        res = manual_annotation(idx)
        if res == -1:
            return -1

# the UI (?) part of the manual annotation
def manual_annotation(idx, inp="."):
    while inp != "":
        sample = samples[idx]
        print("Source sentence: ", sample['source'])
        print("Reference: ", sample['reference'])
        print("Good Translation: ", sample['good-translation'])
        print("Incorrect Translation: ", sample['incorrect-translation'])
        inp = input("Enter the incorrect translation with the < and > to show the error spans (exit to stop, skip to skip): \n")
        bad = inp
        if bad == "exit":
            return -1
        if bad == "skip":
            return 1
        inp = input("Enter the correct/reference translation with the < and > to show the error spans (exit to stop, skip to skip): \n")
        good = inp
        if good == "exit":
            return -1
        if good == "skip":
            return 1
        change = calculate_change(good, bad, sample)
        print("Annotation: ", change)
        inp = input("\n To accept it press enter or to annotate again enter any other string: ")
        if inp == "":
            sample['annotation'] = change
            sample['method'] = "manual annotation"
            annotations[idx] = sample
    return annotations[idx]

# given a manually annotated sample (where there are <> in incorrect and good/reference sentences)
# calculate the character spans in the original sentences and return the change in our annotation format
def calculate_change(good, bad, sample):  
    bad_id = 0
    span = False # False is when we are not inside a span, True is inside a span
    change = []
    for i, c in enumerate(bad):
        if c == "<":
            if span:
                logger.error("< not closed. Try again.\n")
                return manual_annotation(".", sample)
            else:
                start = bad_id
                start_annotate = i
                bad_id -= 1
                span = True
        elif c == ">":
            if not span:
                logger.error("No opening < Try again.\n")
                return manual_annotation(".", sample)
            else:
                change.append({"in_good":None, 
                    "in_bad":{'token_index':None, 
                    'character_span':(start,bad_id), 
                               'token':bad[start_annotate+1:i]}})
                bad_id -= 1
                span = False
        bad_id += 1
    good_id = 0
    span = False # False is when we are not inside a span, True is inside a span
    for i, c in enumerate(good):
        if c == "<":
            if span:
                logger.error("< not closed. Try again.\n")
                return manual_annotation(".", sample)
            else:
                start = good_id
                start_annotate = i
                good_id -= 1
                span = True
        elif c == ">":
            if not span:
                logger.error("No opening < Try again.\n")
                return manual_annotation(".", sample)
            else:
                change.append({"in_good":{'token_index':None, 
                    'character_span':(start,good_id), 
                               'token':good[start_annotate+1:i]}, 
                    "in_bad":None})
                good_id -= 1
                span = False
        good_id += 1
    return change

# process given sample, annotate or do manual annotation (only in the annotations.ipynb, in process_dataset.py only automatic annotation)
def process_sample(idx, sample, manual=False, detokenize=False):
    if phenomena[sample["phenomena"]] == 'mixed_flexible':
        good_og = ref_or_good(sample["reference"], sample["good-translation"], sample["incorrect-translation"])
    elif phenomena[sample["phenomena"]] == 'REF_flexible':
        good_og = sample["reference"]
    else:
        good_og = sample["good-translation"]
    bad_og = sample["incorrect-translation"]
    # if detokenize we just annotate the detokenized sentences, then map the character span back to the original sentence
    # in the standardize_annotation function in annotation_utilities.py
    if detokenize:
        try:
            good, good_map = detokenize_text(good_og, lang=sample["langpair"].split('-')[1])
            bad, bad_map = detokenize_text(bad_og, lang=sample["langpair"].split('-')[1])
            maps = (good_map, bad_map)
        except:
            good, bad = good_og, bad_og
            maps = None
    else:
        good, bad = good_og, bad_og
        maps = None # the standardize_annotation function will understand that it does not need to revert detokenization 
        # if maps parameter is None.
    originals = (good_og, bad_og)
    
    if phenomena[sample["phenomena"]] == 'add-omit':
        try:
            change = diff_char_level(good, bad)
            if len(change) == 0:
                logger.warning('No change in id {}'.format(idx))
                stats[sample["phenomena"]]["no_change"].append((idx, sample['langpair']))
            else:
                stats[sample["phenomena"]]["success"] += 1
                change = standardize_annotation(change, good, bad, maps, originals)
            sample['annotation'] = change
            sample['method'] = phenomena[sample["phenomena"]]
            annotations[idx] = sample
        except:
            logger.warning('error in char level annotate, id {}'.format(idx))
            stats[sample["phenomena"]]["error"].append((idx, sample['langpair']))

    elif phenomena[sample["phenomena"]] == 'annotate_word':
        try:
            change = annotate_word(good, bad)
            if len(change) == 0:
                logger.warning('No change in id {}'.format(idx))
                stats[sample["phenomena"]]["no_change"].append((idx, sample['langpair']))
            else:
                stats[sample["phenomena"]]["success"] += 1
                change = standardize_annotation(change, good, bad, maps, originals)
            sample['annotation'] = change
            sample['method'] = phenomena[sample["phenomena"]]
            annotations[idx] = sample
        except:
            logger.warning('error in word level annotate, id {}'.format(idx))
            stats[sample["phenomena"]]["error"].append((idx, sample['langpair']))

    elif phenomena[sample["phenomena"]] in ['diff_flexible', 'REF_flexible', 'mixed_flexible']:
        g, g_spans = tokenize(good)
        b, b_spans = tokenize(bad)

        # special treatment to japanese chinese and thailandish because they don't use spaces, so can't be split            
        if sample['langpair'][-2:] not in ['ja', 'zh', 'th']:      
            if len(g) == len(b):   # if there are multiple one word replacements
                change = diff(g, g_spans, b, b_spans, phenomena="replacement")
            if len(g) != len(b) or len(change) == 0:
                try:
                    change = diff_flexible(good, g, g_spans, bad, b, b_spans)
                    if len(change) == 0 and good != bad:
                        change = diff_char_level(good, bad) 
                except:
                    logger.warning('error in id {}'.format(idx))
                    stats[sample["phenomena"]]["error"].append((idx, sample['langpair']))
            if len(change) == 0:
                logger.warning('No change in id {}'.format(idx,g,b,change))
                stats[sample["phenomena"]]["no_change"].append((idx, sample['langpair']))
            elif len(change) != 0 and ((change[0]['in_good'] != None and len(change[0]['in_good']['token']) > 50) or (change[0]['in_bad'] != None and len(change[0]['in_bad']['token']) > 50)):
                logger.warning('check this - too long: %s' %idx)
                stats[sample["phenomena"]]["too_long"].append((idx, sample['langpair']))
            else:
                stats[sample["phenomena"]]["success"] += 1
                change = standardize_annotation(change, good, bad, maps, originals)
            sample['annotation'] = change
            sample['method'] = phenomena[sample["phenomena"]]
            annotations[idx] = sample  
        else:
            try:
                change = diff_char_level(good, bad) 
                if len(change) == 0 and good != bad:
                    logger.warning('No change in id {}'.format(idx,g,b,change))
                    stats[sample["phenomena"]]["no_change"].append((idx, sample['langpair']))
                elif len(change) != 0 and ((change[0]['in_good'] != None and len(change[0]['in_good']['token']) > 30) or (change[0]['in_bad'] != None and len(change[0]['in_bad']['token']) > 30)):
                    logger.warning('check this - too long: %s' %idx)
                    stats[sample["phenomena"]]["too_long"].append((idx, sample['langpair']))
                else:
                    stats[sample["phenomena"]]["success"] += 1
                    change = standardize_annotation(change, good, bad, maps, originals)
                sample['annotation'] = change
                sample['method'] = phenomena[sample["phenomena"]]
                annotations[idx] = sample
            except: 
                logger.warning('error in id {}'.format(idx))
                stats[sample["phenomena"]]["error"].append((idx, sample['langpair']))

    elif phenomena[sample["phenomena"]] == 'units':
        try:
            g, b, change = annotate_units(good,bad)
            if len(change) == 0 and g != b:
                logger.warning('No change in id {}, \ng: {}, \nb: {},\nr: {}'.format(idx, g, b))
                stats[sample["phenomena"]]["no_change"].append((idx, sample['langpair']))
            elif len(change) > 1:
                logger.warning('Multiple changes in {} id {}'.format(sample["phenomena"], idx))
                stats[sample["phenomena"]]["other"].append((idx, sample['langpair']))
            else:
                stats[sample["phenomena"]]["success"] += 1
                change = standardize_annotation(change, good, bad, maps, originals)
            sample['annotation'] = change
            sample['method'] = phenomena[sample["phenomena"]]
            annotations[idx] = sample  
        except: 
            logger.warning('error in id {}'.format(idx))
            stats[sample["phenomena"]]["error"].append((idx, sample['langpair']))

    elif phenomena[sample["phenomena"]] == 'swap':
        try:
            change = annotate_swap_word_lvl(good,bad)
            if len(change) < 2 and good != bad:
                logger.warning('No change in id {}, \ng: {}, \nb: {}'.format(idx, good, bad))
                stats[sample["phenomena"]]["no_change"].append((idx, sample['langpair']))
            elif change[0]['in_good'] != None and change[1]['in_good'] != None and change[0]['in_good'] == change[1]['in_good']:
                logger.warning('check this: %s - swapped words are the same!' %idx)
                stats[sample["phenomena"]]["other"].append((idx, sample['langpair']))
            elif (change[0]['in_good'] != None and len(change[0]['in_good']['token']) > 50) or (change[0]['in_bad'] != None and len(change[0]['in_bad']['token']) > 50):
                logger.warning('check this: %s' %idx)
                stats[sample["phenomena"]]["too_long"].append((idx, sample['langpair']))
            else:
                stats[sample["phenomena"]]["success"] += 1
                change = standardize_annotation(change, good, bad, maps, originals)
            sample['annotation'] = change
            sample['method'] = phenomena[sample["phenomena"]]
            annotations[idx] = sample
        except: 
            logger.warning('error in id {}'.format(idx))
            stats[sample["phenomena"]]["error"].append((idx, sample['langpair']))

    elif phenomena[sample["phenomena"]] == 'date':
        try:
            change = diff_dates(good,bad)
            stats[sample["phenomena"]]["success"] += 1
            change = standardize_annotation(change, good, bad, maps, originals)
            sample['annotation'] = change
            sample['method'] = phenomena[sample["phenomena"]]
            annotations[idx] = sample
        except: 
            logger.warning('error in id {}'.format(idx))
            stats[sample["phenomena"]]["error"].append((idx, sample['langpair']))
    elif phenomena[sample['phenomena']] == 'whole_sentence':
        change = whole_sentence(good, bad)
        stats[sample["phenomena"]]["success"] += 1
        change = standardize_annotation(change, good, bad, maps, originals)
        sample['annotation'] = change
        sample['method'] = phenomena[sample["phenomena"]]
        annotations[idx] = sample
    if manual:
        res = manual_annotation_io(idx)
        if res == 1:  # SKIPPING
            return 1 
        # if exit, first save a new annotations file to save progress and then exit
        if res == -1:
            with open(checkpoint, "w+") as f:
                json.dump(annotations, f, indent=2, ensure_ascii=False)  # encode dict into JSON
            return -1
    return 1  # 1 for success
        
def process_phenomena(samples, manual=False, detokenize=False):
    for idx,sample in tqdm(samples.items()):
        # here don't worry about stats - it will be probably completely wrong
        if idx not in annotations.keys():
            stats[sample["phenomena"]]["total"] += 1
            
            # check if it was annotated before
            res = check_seen_before(sample, annotations)
            if res != None:
                sample['annotation'] = res[0]
                sample['method'] = res[1]
                annotations[idx] = sample
            else:
                try:
                    res = process_sample(idx, sample, manual, detokenize)
                except:
                    logger.error(idx)
                if res == -1:
                    return -1
    # save all annotations after finished
    with open(checkpoint, "w+") as f:
        json.dump(annotations, f, indent=2, ensure_ascii=False)  # encode dict into JSON

enter the phenomena: test


INFO:logger:Loading the test dataset...
INFO:logger:Test dataset loaded.
INFO:logger:Path /mnt/c/Users/user/OneDrive/Masaüstü/work/ACES_private/challenge_set_annotation/manual_annotations/annotated_checkpoint_test.txt already exists. Loading..
INFO:logger:Creating new stats.txt file at /mnt/c/Users/user/OneDrive/Masaüstü/work/ACES_private/challenge_set_annotation/ACES_private/challenge_set_annotation/stats.txt
INFO:logger:READY


In [135]:
# run this cell to start from the beginning - not from a checkpoint (you will lose the prev checkpoint tho)
# annotations = dict()

In [2]:
# THE ACTUAL PART
logger.setLevel(logging.INFO)
process_phenomena(samples, manual=True, detokenize=True)

  0%|                                                                                                                                   | 0/100 [00:00<?, ?it/s]

-----> For this sample we can compare the Incorrect translation with either Reference or Good translation.

ID:  1100
Source sentence:  Susan made the decision of getting in an airplane instead of train to go to Miami because it travels fast.
Reference:  Susan a pris la décision de monter dans un avion au lieu du train pour se rendre à Miami parce que l' avion voyage vite.
Good Translation:  Susan a pris la décision de monter dans un avion au lieu du train pour aller à Miami parce qu'il voyage rapidement.
Incorrect Translation:  Susan a pris la décision de monter dans un avion au lieu du train pour se rendre à Miami parce que le train voyage vite.
Suggested annotation:
[{'in_good': {'token_index': [21], 'character_span': (99, 106), 'token': "l'avion"}, 'in_bad': {'token_index': [21, 22], 'character_span': (99, 107), 'token': 'le train'}}] 

To accept the suggested annotation click on enter. To skip this one enter skip. To exit enter exit and enter anything else to manually annotate:a
S

  6%|███████▍                                                                                                                   | 6/100 [00:25<06:46,  4.33s/it]


-1

In [5]:
sample = samples["1100"]

In [6]:
sample

{'source': 'Susan made the decision of getting in an airplane instead of train to go to Miami because it travels fast.',
 'good-translation': "Susan a pris la décision de monter dans un avion au lieu du train pour aller à Miami parce qu'il voyage rapidement.",
 'incorrect-translation': 'Susan a pris la décision de monter dans un avion au lieu du train pour se rendre à Miami parce que le train voyage vite.',
 'reference': "Susan a pris la décision de monter dans un avion au lieu du train pour se rendre à Miami parce que l' avion voyage vite.",
 'phenomena': 'coreference-based-on-commonsense',
 'langpair': 'en-fr'}

In [13]:
bad_map

{0: 0,
 1: 1,
 2: 2,
 3: 3,
 4: 4,
 5: 5,
 6: 6,
 7: 7,
 8: 8,
 9: 9,
 10: 10,
 11: 11,
 12: 12,
 13: 13,
 14: 14,
 15: 15,
 16: 16,
 17: 17,
 18: 18,
 19: 19,
 20: 20,
 21: 21,
 22: 22,
 23: 23,
 24: 24,
 25: 25,
 26: 26,
 27: 27,
 28: 28,
 29: 29,
 30: 30,
 31: 31,
 32: 32,
 33: 33,
 34: 34,
 35: 35,
 36: 36,
 37: 37,
 38: 38,
 39: 39,
 40: 40,
 41: 41,
 42: 42,
 43: 43,
 44: 44,
 45: 45,
 46: 46,
 47: 47,
 48: 48,
 49: 49,
 50: 50,
 51: 51,
 52: 52,
 53: 53,
 54: 54,
 55: 55,
 56: 56,
 57: 57,
 58: 58,
 59: 59,
 60: 60,
 61: 61,
 62: 62,
 63: 63,
 64: 64,
 65: 65,
 66: 66,
 67: 67,
 68: 68,
 69: 69,
 70: 70,
 71: 71,
 72: 72,
 73: 73,
 74: 74,
 75: 75,
 76: 76,
 77: 77,
 78: 78,
 79: 79,
 80: 80,
 81: 81,
 82: 82,
 83: 83,
 84: 84,
 85: 85,
 86: 86,
 87: 87,
 88: 88,
 89: 89,
 90: 90,
 91: 91,
 92: 92,
 93: 93,
 94: 94,
 95: 95,
 96: 96,
 97: 97,
 98: 98,
 99: 99,
 100: 100,
 101: 101,
 102: 102,
 103: 103,
 104: 104,
 105: 105,
 106: 106,
 107: 107,
 108: 108,
 109: 109,
 110: 110,

In [9]:
change

[{'in_good': {'token_index': [21],
   'character_span': (99, 106),
   'token': "l'avion"},
  'in_bad': {'token_index': [21, 22],
   'character_span': (99, 107),
   'token': 'le train'}}]

here


[{'in_good': {'token_index': [21],
   'character_span': (99, 106),
   'token': "l'avion"},
  'in_bad': {'token_index': [21, 22],
   'character_span': (99, 107),
   'token': 'le train'}}]

In [8]:
idx = 1100

good_og = ref_or_good(sample["reference"], sample["good-translation"], sample["incorrect-translation"])
bad_og = sample["incorrect-translation"]

good, good_map = detokenize_text(good_og, lang=sample["langpair"].split('-')[1])
bad, bad_map = detokenize_text(bad_og, lang=sample["langpair"].split('-')[1])
maps = (good_map, bad_map)

originals = (good_og, bad_og)
g, g_spans = tokenize(good)
b, b_spans = tokenize(bad)

# special treatment to japanese chinese and thailandish because they don't use spaces, so can't be split            
if sample['langpair'][-2:] not in ['ja', 'zh', 'th']:      
    if len(g) == len(b):   # if there are multiple one word replacements
        change = diff(g, g_spans, b, b_spans, phenomena="replacement")
    if len(g) != len(b) or len(change) == 0:
        try:
            change = diff_flexible(good, g, g_spans, bad, b, b_spans)
            if len(change) == 0 and good != bad:
                change = diff_char_level(good, bad) 
        except:
            logger.warning('error in id {}'.format(idx))
            stats[sample["phenomena"]]["error"].append((idx, sample['langpair']))
    if len(change) == 0:
        logger.warning('No change in id {}'.format(idx,g,b,change))
        stats[sample["phenomena"]]["no_change"].append((idx, sample['langpair']))
    elif len(change) != 0 and ((change[0]['in_good'] != None and len(change[0]['in_good']['token']) > 50) or (change[0]['in_bad'] != None and len(change[0]['in_bad']['token']) > 50)):
        logger.warning('check this - too long: %s' %idx)
        stats[sample["phenomena"]]["too_long"].append((idx, sample['langpair']))
    else:
        stats[sample["phenomena"]]["success"] += 1
        

In [2]:
from annotation_utilities import *


In [6]:
change

[{'in_good': {'token_index': [21],
   'character_span': (99, 106),
   'token': "l'avion"},
  'in_bad': {'token_index': [21, 22],
   'character_span': (99, 107),
   'token': 'le train'}}]

In [17]:
change_new

[{'in_good': {'token_index': [21],
   'character_span': (99, 107),
   'token': "l' avion"},
  'in_bad': {'token_index': [21],
   'character_span': (99, 107),
   'token': 'le train'}}]

In [20]:
standardize_annotation(change, good, bad, maps, originals)

UnboundLocalError: cannot access local variable 'change_new' where it is not associated with a value

In [21]:
import copy
def standardize_annotation(change, good, bad, maps=None, original=None):
    change_tmp = copy.deepcopy(change)
    if maps != None:
        good_mapping, bad_mapping = maps[0], maps[1]
        good_og, bad_og = original[0], original[1]
        for c in change_tmp:
            c["in_good"]['character_span'] = (good_mapping[c["in_good"]['character_span'][0]], good_mapping[c["in_good"]['character_span'][1]])
            c["in_bad"]['character_span'] = (bad_mapping[c["in_bad"]['character_span'][0]], bad_mapping[c["in_bad"]['character_span'][1]])
            c["in_good"]['token'] = good_og[c["in_good"]['character_span'][0]:c["in_good"]['character_span'][1]]
            c["in_bad"]['token'] = bad_og[c["in_bad"]['character_span'][0]:c["in_bad"]['character_span'][1]]
        
    skip = False
    for c in change:
        if (c['in_good'] != None and (c['in_good']['token_index'] == None or (type(c['in_good']['token_index'])==list and len(c['in_good']['token_index']) > 1)))\
        or (c['in_bad'] != None and (c['in_bad']['token_index'] == None or (type(c['in_bad']['token_index'])==list and len(c['in_bad']['token_index']) > 1)))\
        or c['in_good'] == None or c['in_bad'] == None:
            skip = True
            logger.debug("first check")
            break
    if skip:   # if skipping then change all the integer token indices to lists
        for c in change:
            if c['in_good'] != None and c['in_good']['token_index'] != None and type(c['in_good']['token_index']) != list:
                c['in_good']['token_index'] = [c['in_good']['token_index']]
            if c['in_bad'] != None and c['in_bad']['token_index'] != None and type(c['in_bad']['token_index']) != list:
                c['in_bad']['token_index'] = [c['in_bad']['token_index']]
        print("here")
        return change
    good_tokens = []
    bad_tokens = []
    good_span = ()   # char span
    bad_span = ()
    change_new = []
    for c in change:
        g = c['in_good']
        b = c['in_bad']
        if type(g['token_index']) == list:
            g['token_index'] = g['token_index'][0]
        if type(b['token_index']) == list:
            b['token_index'] = b['token_index'][0]
        if len(good_tokens) == 0 and len(bad_tokens) == 0:
            good_tokens.append(g['token_index'])
            bad_tokens.append(b['token_index'])
            good_span = g['character_span']
            bad_span = b['character_span']
        elif g['token_index'] == good_tokens[-1] + 1 and b['token_index'] == bad_tokens[-1] + 1:
            good_tokens.append(g['token_index'])
            bad_tokens.append(b['token_index'])
            good_span = (good_span[0], g['character_span'][1])
            bad_span = (bad_span[0], b['character_span'][1])
        else:
            change_new.append({'in_good': {'token_index': good_tokens,
                        'character_span': good_span,
                        'token': good[good_span[0]:good_span[1]]}, 
                    'in_bad': {'token_index': bad_tokens,
                        'character_span': bad_span,
                        'token': bad[bad_span[0]:bad_span[1]]}})
            good_tokens = [g['token_index']]
            bad_tokens = [b['token_index']]
            good_span = g['character_span']
            bad_span = b['character_span']

    change_new.append({'in_good': {'token_index': good_tokens,
                        'character_span': good_span,
                        'token': good[good_span[0]:good_span[1]]}, 
                    'in_bad': {'token_index': bad_tokens,
                        'character_span': bad_span,
                        'token': bad[bad_span[0]:bad_span[1]]}})
    
    
    return change_new

In [ ]:
# After this cell there are extra stuff - no need to run

In [109]:
# To run normalization over the already annotated samples in coreference-based-on-commonsense
# No need to do that for manual annotation!
for idx in annotations:
    sample = annotations[idx]
    good, _, _ = ref_or_good(sample["reference"], sample["good-translation"], sample["incorrect-translation"])
    bad = sample['incorrect-translation']
    change = sample['annotation']
    try:
        sample['annotation'] = standardize_annotation(change, good, bad)
    except:
        print(idx)

In [112]:
with open(checkpoint, "w+") as f:
    json.dump(annotations, f, indent=2, ensure_ascii=False)  # encode dict into JSON

In [6]:
# to count the number of samples in any phenomena
# No need to do that for manual annotation!
count = 0
for sample in dataset["train"]:
    if sample["phenomena"] in ["lexical-overlap", 'xnli-omission-neutral', 'xnli-omission-contradiction', 'xnli-addition-neutral', 'xnli-addition-contradiction']:
        count += 1